# MTC Network Simulation (100-MTD Network)

Extended MTC network simulation programme created to generate binary transmission time series for 100 MTDs where 5-MTDs exhibits periodic transmissions, other 5-MTDs exhibits random transmissions and rest of MTDs exhibits event-driven transmissions. Every event-driven transmitting MTD have a causal relationship with periodic transmitting MTD or random transmitting MTD. The periodic transmitting MTDs and random transmitting MTDs act as event triggering MTD. Therefore, we assume whenever IoT event occurred this event triggering MTD get affect by the event before other MTDs exhibits event-driven transmission. This assumption hold because of the geographically propagating nature of IoT events.

## Import Python Library

In [1]:
import sys
import numpy as np
import pandas as pd
import random as rd
from random import choices

## Python Progress Bar

In [2]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    file.write("\n")
    file.flush()

## Random Transmission
Random transmission can be happen at any given time step complete randomly. In other work for any given time step probability of transmission is 0.5 and probability of being silent is 0.5. 

In [3]:
dataset = []
for i in progressbar(range(0,5)):
    Hb = choices((0,1), (0.5,0.5),k=60000)
    dataset.append(Hb)
dataset = np.array(dataset)

[############################################################] 5/5


In [4]:
dataset.shape

(5, 60000)

## Periodic Transmission
Periodic transmission happens based on fixed repeating probability distribution where transmission probability is periodically set to a value within interval [0, 1] throughout the transmission time series. As an example transmission probability of time step (10N +X) is set to fix P probability where N is natural number and X = [0, 9] and P = [0, 1]. 

In [5]:
for i in progressbar(range(0,5)):
    DS = []
    for k in range(0,10):
        TXp = np.random.randint(0,11)/10
        Hb = choices((0,1), (TXp,1-TXp),k=6000)
        DS.append(Hb)
    DS = np.array(DS)
    DS = np.transpose(DS)
    DS = DS.reshape(1,-1)
    dataset = np.vstack([dataset, DS])

[############################################################] 5/5


In [6]:
dataset.shape

(10, 60000)

## Event Driven Transmission
Event-driven transmission occur with P correlated probability to periodic and random transmitting MTD after X time step to event is triggered where P = [0.8, 1] and T = [0, 9]. 

In [7]:
for i in progressbar(range(0,10)):
    Ep = rd.sample(range(1, 10), 9)
    for k in range(0, 9):
        X = [[0]]*Ep[k]
        Ia = np.random.randint(8,11)/10
        population = [1,0]
        weights = [Ia,1-Ia]
        for j in range(Ep[k], 60000):
            if dataset[i][j-Ep[k]] == 1:
                MTD = choices(population, weights)
            if dataset[i][j-Ep[k]] == 0:
                MTD = [0]
            X.append(MTD)
        X = np.array(X)
        X = X.reshape(1,-1)
        dataset = np.vstack([dataset, X])

[############################################################] 10/10


In [9]:
dataset.shape

(100, 60000)

# Data Pre-processing

## Shuffle dataset np array by row only, keep column order unchanged

In [10]:
np.random.shuffle(dataset)

In [11]:
dataset.shape

(100, 60000)

## Split dataset for training, validation and testing

In [12]:
training = dataset[:,:20000]
validation = dataset[:,20000:40000]
test = dataset[:,40000:60000]

## Save dataset as csv files

In [13]:
training = pd.DataFrame(training)
validation = pd.DataFrame(validation)
test = pd.DataFrame(test)

training.to_csv('training.csv', index=False)
validation.to_csv('validation.csv', index=False)
test.to_csv('test.csv', index=False)

## Load dataset from csv files

In [14]:
train = pd.read_csv('training.csv')
validation  = pd.read_csv('validation.csv')
test  = pd.read_csv('test.csv')

## Training Data Pre-processing 

In [15]:
X_TRAIN = [[[]*1]*10]*19990
X_TRAIN = np.array(X_TRAIN)
Y_TRAIN = [[]*1]*19990
Y_TRAIN = np.array(Y_TRAIN)
for j in progressbar(range(0,100)):
    X_train = []
    Y_train = []
    t = np.array(train.iloc[j, :]).reshape(-1,1)
    for i in range(10, 20000):
        X_train.append(t[i-10:i, 0])
        Y_train.append(t[i, 0])
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    Y_train = np.reshape(Y_train, (Y_train.shape[0],1))
    X_TRAIN = np.concatenate((X_TRAIN, X_train),axis=2)
    Y_TRAIN = np.concatenate((Y_TRAIN, Y_train),axis=1)

[############################################################] 100/100


Independed training-feature of the matrix = X_TRAIN

In [16]:
X_TRAIN.shape

(19990, 10, 100)

Depended training-feature of the matrix = Y_TRAIN

In [17]:
Y_TRAIN.shape

(19990, 100)

## Validation Data Pre-processing 

In [18]:
X_VAL = [[[]*1]*10]*19990
X_VAL = np.array(X_VAL)
Y_VAL = [[]*1]*19990
Y_VAL = np.array(Y_VAL)
for j in progressbar(range(0,100)):
    X_validation = []
    Y_validation = []
    t = np.array(train.iloc[j, :]).reshape(-1,1)
    for i in range(10, 20000):
        X_validation.append(t[i-10:i, 0])
        Y_validation.append(t[i, 0])
    X_validation, Y_validation = np.array(X_validation), np.array(Y_validation)
    X_validation = np.reshape(X_validation, (X_validation.shape[0], X_validation.shape[1], 1))
    Y_validation = np.reshape(Y_validation, (Y_validation.shape[0],1))
    X_VAL = np.concatenate((X_VAL, X_validation),axis=2)
    Y_VAL = np.concatenate((Y_VAL, Y_validation),axis=1)

[############################################################] 100/100


Independed validation-feature of the matrix = X_VAL

Depended validation-feature of the matrix = Y_VAL

## Test Data Pre-processing 

In [19]:
X_TEST = [[[]*1]*10]*19990
X_TEST = np.array(X_TEST)
Y_TEST = [[]*1]*19990
Y_TEST = np.array(Y_TEST)
for j in progressbar(range(0,100)):
    X_test = []
    Y_test = []
    t = np.array(test.iloc[j, :]).reshape(-1,1)
    for i in range(10, 20000):
        X_test.append(t[i-10:i, 0])
        Y_test.append(t[i, 0])
    X_test, Y_test = np.array(X_test), np.array(Y_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    Y_test = np.reshape(Y_test, (Y_test.shape[0],1))
    X_TEST = np.concatenate((X_TEST, X_test),axis=2)
    Y_TEST = np.concatenate((Y_TEST, Y_test),axis=1)

[############################################################] 100/100


## Save Pre-processed data (prepare for S3 bucket uploading)

In [ ]:
np.savez('train_restruct', dat_tr=X_TRAIN, lbl_tr=Y_TRAIN)
np.savez('valid_restruct', dat_ts=X_VAL, lbl_ts=Y_VAL)
np.savez('test_restruct', dat_ts=X_TEST, lbl_ts=Y_TEST)